In [1]:
import numpy as np
import cv2

In [2]:
import tensorflow as tf

In [3]:
from tensorflow import keras

In [4]:
from tensorflow.keras import Sequential

In [5]:
from tensorflow.keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [6]:
train_dir = 'data/train'
val_dir = 'data/test'
train_data_gen = ImageDataGenerator(rescale = 1./255)
val_data_gen = ImageDataGenerator(rescale = 1./255)


In [7]:
train_generator = train_data_gen.flow_from_directory(train_dir,
                                                    target_size = (48,48),
                                                    batch_size = 64,
                                                     color_mode = "grayscale",
                                                     class_mode = 'categorical'
                                                    )

Found 28709 images belonging to 7 classes.


In [8]:
validation_generator = val_data_gen.flow_from_directory(val_dir,
                                                       target_size = (48,48),
                                                       batch_size = 64,
                                                       color_mode='grayscale',
                                                       class_mode = 'categorical'
                                                       )

Found 7178 images belonging to 7 classes.


In [15]:
# Building the cnn network

In [9]:
emotion_model = Sequential()

In [10]:
emotion_model.add(Conv2D(32,kernel_size =(3,3),activation = 'relu',input_shape = (48,48,1)))

In [11]:
emotion_model.add(Conv2D(64,kernel_size =(3,3),activation = 'relu',input_shape = (48,48,1)))

In [12]:
emotion_model.add(MaxPooling2D(pool_size = (2,2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128,kernel_size = (3,3),activation = 'relu'))

In [13]:
emotion_model.add(MaxPooling2D(pool_size = (2,2)))
emotion_model.add(Conv2D(128,kernel_size = (3,3),activation = 'relu'))
emotion_model.add(MaxPooling2D(pool_size = (2,2)))
emotion_model.add(Dropout(0.25))

In [14]:
emotion_model.add(Flatten())
emotion_model.add(Dense(1024,activation = 'relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7,activation = 'softmax'))

In [15]:
# compile and train the model
emotion_model.compile(loss='categorical_crossentropy',optimizer=Adam(lr = 0.0001,decay = 1e-6),
                     metrics=['accuracy'])

In [16]:
emotional_model_info = emotion_model.fit(
train_generator,steps_per_epoch= 28709//64,
epochs = 50,
validation_data = validation_generator,
validation_steps = 7178//64
)
# emotional_model_info = emotion_model.fit(
# train_generator,
# epochs = 50,
# validation_data = validation_generator,
# )


Epoch 1/50
448/448 [==============================] - 22s 48ms/step - loss: 1.8074 - accuracy: 0.2539 - val_loss: 1.7255 - val_accuracy: 0.3273
Epoch 2/50
448/448 [==============================] - 11s 26ms/step - loss: 1.6391 - accuracy: 0.3600 - val_loss: 1.5511 - val_accuracy: 0.4125
Epoch 3/50
448/448 [==============================] - 12s 26ms/step - loss: 1.5325 - accuracy: 0.4071 - val_loss: 1.4674 - val_accuracy: 0.4415
Epoch 4/50
448/448 [==============================] - 12s 26ms/step - loss: 1.4610 - accuracy: 0.4405 - val_loss: 1.4355 - val_accuracy: 0.4533
Epoch 5/50
448/448 [==============================] - 12s 26ms/step - loss: 1.4053 - accuracy: 0.4656 - val_loss: 1.3593 - val_accuracy: 0.4823
Epoch 6/50
448/448 [==============================] - 13s 29ms/step - loss: 1.3563 - accuracy: 0.4823 - val_loss: 1.3216 - val_accuracy: 0.4996
Epoch 7/50
448/448 [==============================] - 12s 26ms/step - loss: 1.3140 - accuracy: 0.4987 - val_loss: 1.2857 - val_accuracy:

In [17]:
emotion_model.save_weights('model.h5')


In [27]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15885264226605189957
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 2312281272197040048
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 9303167960032795397
physical_device_desc: "device: XLA_GPU device"
]


In [29]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  0


In [18]:
cv2.ocl.setUseOpenCL(False)
emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if not ret:
        break
    bounding_box = cv2.CascadeClassifier('C:\\Users\Bhojwani\Anaconda3\Lib\site-packages\cv2\data\haarcascade_frontalface_default.xml')
    gray_frame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    num_faces = bounding_box.detectMultiScale(gray_frame,scaleFactor=1.3, minNeighbors=5)
    for (x, y, w, h) in num_faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (255, 0, 0), 2)
        roi_gray_frame = gray_frame[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)
        emotion_prediction = emotion_model.predict(cropped_img)
        maxindex = int(np.argmax(emotion_prediction))
        cv2.putText(frame, emotion_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.imshow('Video', cv2.resize(frame,(1200,860),interpolation = cv2.INTER_CUBIC))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break